In [1]:
%load_ext google.cloud.bigquery

In [2]:
import numpy as np 
import pandas as pd
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/shuang/Downloads/bqair-2b59936a9500.json"

# Big query helpers
from google.cloud import bigquery
from bq_helper import BigQueryHelper

# Import plotting libaries
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.io as pio
import geopandas
import shapely

# the Plotly in offline mode we can see map in the notebook.
from plotly.offline import download_plotlyjs,init_notebook_mode, plot,iplot
init_notebook_mode(connected=True)

In [3]:
import plotly
plotly.tools.set_credentials_file(username='cleopathy', api_key='Vt9pQQVXJcV8bqHarLVt')

In [4]:
# getting pm25 data:
QUERY = """
    SELECT
        state_code,
        county_code,
        date_local,
        arithmetic_mean,
        state_name,
        county_name,
        aqi
    FROM
      `bigquery-public-data.epa_historical_air_quality.pm25_frm_daily_summary`
    WHERE
      poc = 1
      AND EXTRACT(YEAR FROM date_local) in (2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017)
    
        """

In [5]:
bq_assistant = BigQueryHelper("bigquery-public-data", "epa_historical_air_quality")
df_pm25 = bq_assistant.query_to_pandas(QUERY) # getting data through big query to pandas
df_pm25['date_local'] = pd.to_datetime(df_pm25['date_local'])
df_pm25['year'] = df_pm25['date_local'].dt.year
df_pm25['month'] = df_pm25['date_local'].dt.month
import calendar
df_pm25['month_name'] = df_pm25['month'].apply(lambda x: calendar.month_abbr[x])
df_pm25['aqi'].fillna(df_pm25['aqi'].mean(), inplace = True)
df_pm25.head()

,state_code,county_code,date_local,arithmetic_mean,state_name,county_name,aqi,year,month,month_name
0,78,010,2004-12-17,2.0,Virgin Islands,St Croix,8.0,2004,12,Dec
1,78,030,2007-02-11,4.5,Virgin Islands,St Thomas,19.0,2007,2,Feb
2,78,010,2006-08-03,19.5,Virgin Islands,St Croix,67.0,2006,8,Aug
3,78,010,2006-08-09,6.2,Virgin Islands,St Croix,26.0,2006,8,Aug
4,78,010,2006-10-14,2.7,Virgin Islands,St Croix,11.0,2006,10,Oct


In [6]:
df_pm25.info() 
df_pm25.drop_duplicates()
df_pm25.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1844133 entries, 0 to 44132
Data columns (total 10 columns):
state_code         object
county_code        object
date_local         datetime64[ns]
arithmetic_mean    float64
state_name         object
county_name        object
aqi                float64
year               int64
month              int64
month_name         object
dtypes: datetime64[ns](1), float64(2), int64(2), object(5)
memory usage: 154.8+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1844133 entries, 0 to 44132
Data columns (total 10 columns):
state_code         object
county_code        object
date_local         datetime64[ns]
arithmetic_mean    float64
state_name         object
county_name        object
aqi                float64
year               int64
month              int64
month_name         object
dtypes: datetime64[ns](1), float64(2), int64(2), object(5)
memory usage: 154.8+ MB


In [8]:
tg_pm25 = df_pm25
tg_pm25_avg = tg_pm25.groupby('state_name')['aqi'].mean()
tg_pm25_avg = tg_pm25_avg.reset_index()
tg_pm25_avg.aqi = tg_pm25_avg.aqi.round()
tg_pm25_avg.head()

,state_name,aqi
0,Alabama,45.0
1,Alaska,34.0
2,Arizona,32.0
3,Arkansas,43.0
4,California,41.0


In [16]:
import sklearn 

In [18]:
from sklearn.model_selection import train_test_split

In [ ]:
#now we would like to predict AQI of pm25 using sklearn,but now we have to use pandas get dummies and one-hot encoder.